In [7]:
import tensorflow as tf


A computational graph is a series of TensorFlow operations arranged into a graph of nodes. Let's build a simple computational graph. Each node takes zero or more tensors as inputs and produces a tensor as an output. One type of node is a constant. Like all TensorFlow constants, it takes no inputs, and it outputs a value it stores internally. We can create two floating point Tensors node1 and node2 as follows:



In [8]:

node1 = tf.constant(6.0, dtype=tf.float32)
node2 = tf.constant(4.0) # also tf.float32 implicitly
print(node1, node2)

Tensor("Const_2:0", shape=(), dtype=float32) Tensor("Const_3:0", shape=(), dtype=float32)


In [9]:
sess = tf.Session()
print(sess.run([node1, node2]))


[6.0, 4.0]


In [10]:
node3 = tf.add(node1, node2)
print("node3:", node3)
print("sess.run(node3):", sess.run(node3))


node3: Tensor("Add_1:0", shape=(), dtype=float32)
sess.run(node3): 10.0


In [11]:
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
adder_node = a + b  # + provides a shortcut for tf.add(a, b)


In [12]:
print(sess.run(adder_node, {a: 3, b: 4.5}))
print(sess.run(adder_node, {a: [1, 3], b: [2, 4]}))


7.5
[ 3.  7.]


In [13]:
add_and_triple = adder_node * 3.
print(sess.run(add_and_triple, {a: 3, b: 4.5}))


22.5


In [14]:
W = tf.Variable([.3], dtype=tf.float32)
b = tf.Variable([-.3], dtype=tf.float32)
x = tf.placeholder(tf.float32)
linear_model = W * x + b


Constants are initialized when you call tf.constant, and their value can never change. By contrast, variables are not initialized when you call tf.Variable. To initialize all the variables in a TensorFlow program, you must explicitly call a special operation as follows:



In [15]:
init = tf.global_variables_initializer()
sess.run(init)


It is important to realize init is a handle to the TensorFlow sub-graph that initializes all the global variables. Until we call sess.run, the variables are uninitialized.

Since x is a placeholder, we can evaluate linear_model for several values of x simultaneously as follows:



In [16]:
print(sess.run(linear_model, {x: [1, 2, 3, 4]}))



[ 0.          0.30000001  0.60000002  0.90000004]


A loss function measures how far apart the current model is from the provided data. We'll use a standard loss model for linear regression, which sums the squares of the deltas between the current model and the provided data. linear_model - y creates a vector where each element is the corresponding example's error delta. We call tf.square to square that error. Then, we sum all the squared errors to create a single scalar that abstracts the error of all examples using tf.reduce_sum:



In [17]:

y = tf.placeholder(tf.float32)
squared_deltas = tf.square(linear_model - y)
loss = tf.reduce_sum(squared_deltas)
print(sess.run(loss, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]}))


23.66


In [18]:
fixW = tf.assign(W, [-5.])
fixb = tf.assign(b, [2.])
sess.run([fixW, fixb])
print(sess.run(loss, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]}))


404.0


just show W*x + b

linear_model = W * x + b



In [19]:
print(sess.run(linear_model, {x: [1, 2, 3, 4]}))


[ -3.  -8. -13. -18.]


what happens about reassigning the variables


In [20]:
#deltas = tf.placeholder(tf.float32)
fixW = tf.assign(W, [-6.])
fixb = tf.assign(b, [3.])

deltas = (linear_model - y) # Wx + b

loss = tf.reduce_sum(squared_deltas)
print(sess.run(loss, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]}))



404.0


In [21]:

print(sess.run(deltas, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]}))

print(deltas)

[ -3.  -7. -11. -15.]
Tensor("sub_1:0", dtype=float32)


In [22]:
print(W)

<tf.Variable 'Variable:0' shape=(1,) dtype=float32_ref>


In [23]:
print(sess.run(W))



[-5.]


In [24]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

In [25]:
sess.run(init) # reset values to incorrect defaults.
for i in range(1000):
  sess.run(train, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]})

print(sess.run([W, b]))


[array([-0.9999969], dtype=float32), array([ 0.99999082], dtype=float32)]
